In [6]:
!pip install -q PyPDF2 sentence-transformers google-generativeai langchain chromadb langchain-community

In [7]:
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import tempfile

In [8]:
# 🔐 Set your Gemini API key here
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    import getpass
    GEMINI_API_KEY = getpass.getpass("Enter your Gemini API key: ")

genai.configure(api_key=GEMINI_API_KEY)

Enter your Gemini API key: ··········


In [9]:
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]


Saving Java_String_Functions_Cheat_Sheet.pdf to Java_String_Functions_Cheat_Sheet.pdf


In [10]:
def extract_text_from_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        content = page.extract_text()
        if content:
            text += content + "\n"
    return text

raw_text = extract_text_from_pdf(pdf_path)
print("✅ PDF text extracted.")


✅ PDF text extracted.


In [11]:
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_text(raw_text)
print(f"🔹 Split into {len(chunks)} chunks.")


🔹 Split into 1 chunks.


In [12]:
def build_chroma_index(chunks):
    docs = [Document(page_content=chunk) for chunk in chunks]
    embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(docs, embedding=embedder, persist_directory=tempfile.mkdtemp())
    return vectorstore

vectorstore = build_chroma_index(chunks)
print("✅ Chroma vector index created.")


/tmp/ipython-input-2155472584.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ Chroma vector index created.


In [13]:
def retrieve_top_k(query, vectorstore, k=4):
    results = vectorstore.similarity_search(query, k=k)
    return [doc.page_content for doc in results]

def ask_gemini(query, k=4):
    top_chunks = retrieve_top_k(query, vectorstore, k)
    context = "\n\n".join(top_chunks)
    prompt = f"""You are a helpful assistant. Answer the question using the context below.

Context:
{context}

Question:
{query}

Answer:"""
    g_model = genai.GenerativeModel("models/gemini-1.5-flash")
    response = g_model.generate_content(prompt)
    return response.text.strip()


In [ ]:
while True:
    question = input("\nAsk a question (or type 'exit'): ")
    if question.lower() == "exit":
        break
    print("\n🤖", ask_gemini(question))



Ask a question (or type 'exit'): what are java functions?

🤖 Based on the provided text, Java functions (in this context, methods of the String and StringBuilder classes) are operations that perform specific actions on strings.  These include finding the length of a string, extracting substrings, modifying case, comparing strings, replacing characters, splitting strings, and appending or inserting into strings.  The examples show how each function is used with its parameters and the resulting output.
